In [1]:
import cobra
model = cobra.io.read_sbml_model('/data1/xpgeng/cross_pathogen/GEM_model/iZ_1308.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
import csv
one_gene = {}
with open('/data1/xpgeng/cross_pathogen/one_two_knockout/' +'iZ_1308_1KO_growth', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) 
    for row in reader:
        #print(len(row))
        if len(row) == 4:            
            one_gene[row[0]] = int(row[3])           
    print(len(one_gene)) 

1308


In [3]:
gene_list = []
for k,v in one_gene.items():
    gene_list.append(k)
print(len(gene_list))

1308


In [4]:
from itertools import combinations
twogene=list(combinations(gene_list,2))
from tqdm import tqdm

In [ ]:
with open('iZ_1308_2KO_growth', mode='a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    firstrow = ['gene, label']
    writer.writerow(firstrow)  

    state_0   = cobra.flux_analysis.pfba(model)      
    growth_0    = round(state_0['BIOMASS_Ec_iJO1366_core_53p95M'], 3)

    for i in tqdm(twogene):   
        with model:                
            try:            
                model.genes.get_by_id(i[0]).knock_out()
                model.genes.get_by_id(i[1]).knock_out()
                state_1   = cobra.flux_analysis.pfba(model)
                growth_1    = round(state_1['BIOMASS_Ec_iJO1366_core_53p95M'], 3)
                if growth_1 < 0.1*growth_0:
                    
                    writer.writerow([i[0], i[1], growth_0, growth_1, 1]) 
                else:
                    writer.writerow([i[0], i[1], growth_0, growth_1, 0])
            except:
                writer.writerow([i, 'fail'])

  0%|                        | 105/854778 [00:50<115:43:03,  2.05it/s]